# Initialization

In [0]:
%run "/Workspace/Users/amberasad0299@gmail.com/databricks_data_lakehouse_project/scripts/silver/silver_util"

In [0]:
import pyspark.sql.functions as F
from pyspark.sql.types import StringType
from pyspark.sql.functions import trim, col

# Read Bronze Table

In [0]:
bronze_table_name = "erp_px_cat_g1v2"
df = read_bronze(spark, bronze_table_name)

# Silver Transformations

## Trimming

In [0]:
df = trim_all_strings(df)

## Normalize Maintenance Flag to Boolean

In [0]:
df = df.withColumn(
    "maintenance",
    F.when(F.upper(col("maintenance")) == "YES", F.lit(True))
     .when(F.upper(col("maintenance")) == "NO", F.lit(False))
     .otherwise(None)
)

## Renaming Columns

In [0]:
RENAME_MAP = {
    "id": "category_id",
    "cat": "category",
    "subcat": "subcategory",
    "maintenance": "maintenance_flag"
}
df = rename_columns(df, RENAME_MAP)

## Sanity Check before Writing

In [0]:
df.limit(10).display()

# Writing Silver Table

In [0]:
silver_table_name = "erp_product_category"
write_silver(df, silver_table_name)

## Sanity Check after Writing

In [0]:
%sql
SELECT * FROM workspace.silver.erp_product_category LIMIT 10